### Definition of ERICs
ERICs are characterized by argumentative, respectful exchanges containing persuasive, informative, and/or sympathetic comments. They tend to stay on topic with the original article and not to contain funny, mean, or sarcastic comments. We found differences between the distribution of annotations made by trained and untrained anno- tators, but high levels of agreement within each group, suggesting that crowdsourcing annotations for this task is reliable.

In [6]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re



In [79]:
df = pd.read_csv('./data/yahoodata/ydata-ynacc-v1_0_expert_annotations.tsv',sep='\t')
df.drop(columns=['sdid','commentindex','headline','url','guid','commentid','timestamp','thumbs-up','thumbs-down',
                'parentid','intendedaudience'],inplace=True)
df.head()


,text,constructiveclass,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,persuasiveness
0,"These things happen , Every job has its dangers.",Constructive,NaN,Positive/respectful,negative,NaN,Disagreement with commenter,Off-topic with article,Not persuasive
1,Sad to hear such a bad thing. Very dangerous j...,Constructive,NaN,Positive/respectful,mixed,NaN,NaN,Off-topic with article,Not persuasive
2,Yes..because too many houses in EU look like t...,Constructive,NaN,Positive/respectful,neutral,Informative,NaN,Off-topic with article,Not persuasive
3,"I am frankly quite SICK of the phrase ""shoved ...",Not constructive,Agreement throughout,Off-topic/digression,negative,Mean,NaN,Off-topic with article,Persuasive
4,"Ya, I always wonder why the conservatives are ...",Not constructive,Agreement throughout,Off-topic/digression,neutral,Sarcastic,Agreement with commenter,Off-topic with article,Not persuasive


In [85]:
df.topic.value_counts()

Off-topic with article                                               6909
Personal story                                                       1412
Off-topic with conversation,Off-topic with article                    847
Off-topic with conversation                                           365
Off-topic with article,Personal story                                 164
Off-topic with article,NA                                              77
Personal story,NA                                                      73
Off-topic with conversation,NA                                         26
Off-topic with conversation,Off-topic with article,Personal story      14
Off-topic with conversation,Personal story                             10
Off-topic with conversation,Off-topic with article,NA                   4
Off-topic with article,Personal story,NA                                1
Name: topic, dtype: int64

In [84]:
df[df.tone == 'Informative'].iloc[1].text

'I know this was probably the best thing that ever happened to the boy but let me give you guys a warning... As awesome as it would be to bang your hot teacher there is a big downside. Here it goes... Yes, if the relationship is found out she\'ll go away and you\'ll be treated like a victim BUT if she gets preggo? The state will start billing you for child support when you turn 18 (if the chick asks for it). That\'s right, your big "victimization?" Well, the state will add insult to (legal) injury. And you don\'t want to be paying 18 years of child support from 18 years old so I suggest that you, at least, keep some condoms handy the next time you feel like being... a "victim." LOL!'

In [91]:
iac = pd.read_csv('./data/yahoodata/ydata-ynacc-v1_0_IAC_annotations.tsv',sep='\t')
turk = pd.read_csv('./data/yahoodata/ydata-ynacc-v1_0_turk_annotations.tsv',sep='\t')
turk.columns

Index(['sdid', 'commentindex', 'headline', 'url', 'guid', 'commentid',
       'timestamp', 'thumbs-up', 'thumbs-down', 'text', 'parentid',
       'constructiveclass', 'sd_agreement', 'sd_type'],
      dtype='object')

In [ ]:
# Transform each text into a vector of word counts
vectorizer = CountVectorizer(stop_words="english")

training_features = vectorizer.fit_transform(train_data["text"])    
test_features = vectorizer.transform(test_data["text"])

# Training
model = LinearSVC()
model.fit(training_features, train_data["sentiment"])
y_pred = model.predict(test_features)

# Evaluation
acc = accuracy_score(test_data["sentiment"], y_pred)

print("Accuracy on the IMDB dataset: {:.2f}".format(acc*100))

In [29]:
X = df.text
y = df.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)


3823     And the last excuse your #$%$ had.....because ...
23232    Not sure why you think the battery will only l...
12771    I think what @USERNAME is saying is that the m...
11947    I think these are general statements. My femal...
3342     No, not all "good" PR. I am sure they will los...
                               ...                        
19731    WHAT????  Oh Simpleton.  Never mind self expla...
21259    Newsflash idiot, 99.99% of the black race vote...
19057    People back then were lucky to live past 30. @...
6722     remember it is the criminal doing most gun vio...
12981    The Doobie Brothers? Why not Hootie and the Bl...
Name: text, Length: 1509, dtype: object

array(['negative', 'negative', 'negative', ..., 'negative', 'negative',
       'negative'], dtype='<U8')